In [1]:
import numpy as np
from scipy.stats import poisson


"""
Function to calculate the convolution of two arrays.

Args:
    a (numpy.ndarray): The first array to be convolved.
    b (numpy.ndarray): The second array to be convolved.

Returns:
    numpy.ndarray: The convolution of the two input arrays.
"""
def convolve(a, b):
    
    # Initialize an empty array to store the result.
    c = np.array([])
    
    # Compute the convolution of the two arrays.
    for i in range(len(a)):
        # Get subsets of array expanded to the right.
        a_sub = a[0:i + 1].copy()
        b_sub = b[0:i + 1].copy()
        # Reverse b.
        b_rev = b_sub[::-1]
        # Compute the dot product of a and b_rev.
        c = np.append(c, np.dot(a_sub, b_rev))
    
    for i in range(1,len(a)):
        # Get subsets of array collapse from the right.
        a_sub = a[i:].copy()
        b_sub = b[i:].copy()
        # Reverse b.
        b_rev = b_sub[::-1]
        # Compute the dot product of a and b_rev.
        c = np.append(c, np.dot(a_sub, b_rev))
        
    return c


### Test cases

a = np.array([
    0.4456796414,
    0.160623141,
    0.137676978,
    0.1032577335])

b = np.array([
    0.006737946999,
    0.033689735,
    0.08422433749,
    0.1403738958])

print(convolve(a, b)) # Expected output: [0.00300297 0.0160971  0.04387608 0.08142421]
print(np.convolve(a, b)) # Expected output: [0.00300297 0.0160971  0.04387608 0.08142421]

"""Function to concvolve a distribution with itself n times"""
def convolve_n(a, n):
        
        # array to store the result
        c = np.array([])
        if n == 0:
            c = np.array(np.zeros(len(a)), dtype=np.float64)
            c[0] = 1
            return c
        
        # convolution of a distribution with itself n times is the convolution of the distribution with itself n-1 times
        for i in range(n):
            if i == 0:
                print('i = 0')
                c = a
            else:
                c = np.convolve(c, a)
                
        return c


[0.00300297 0.0160971  0.04387608 0.08142421 0.03762177 0.02802307
 0.01449469]
[0.00300297 0.0160971  0.04387608 0.08142421 0.03762177 0.02802307
 0.01449469]


A schedule with T intervals can have T states. A state of a schedule at interval T is defined by $p^-_t(i)$, the amount of work left at the end of the state at interval $t-1$ and $v_t(i)$, the arriving amount of work in interval $t$.

In [2]:
"""
A schedule class with a constructor and a method to calculate the system states.
"""
class Schedule:
    def __init__(self, x, d, s):
        self.parameters = dict({'x': x, 'd': d, 's': s})
        self.system = dict({
            'p_min': np.zeros((len(self.parameters['x']), len(self.parameters['s'])), dtype=np.float64),
            'v': np.zeros((len(self.parameters['x']), len(self.parameters['s'])), dtype=np.float64)
        })
        self.state = 0  # 0: initial state
        self.system['p_min'][self.state][0] = 1
        self.system['v'][self.state] = self.parameters['s'].copy()
        self.system['p_plus'] = self.system['v'].copy()
        
    def calculate_system_states(self, until = 1):

            while self.state < until:
                """Set state to 1, because state 0 has already been calculated in the constructor."""
                
                self.state += 1
                
                """The probability that the amount of work left in the system equals zero just before state t starts is the probablity that the total amount work in state t-1 was less than or equal to the interval length d."""
                
                self.system['p_min'][self.state][0] = np.sum(self.system['p_plus'][self.state-1][:(self.parameters['d'] + 1)])
                
                """The probability that the amount of work left in the system equals i just before state t starts is the probablity that the total amount work in state t-1 exceeded the interval length d with amount i."""
                
                self.system['p_min'][self.state][1:(-1*self.parameters['d'])] = self.system['p_plus'][self.state-1][(self.parameters['d'] + 1):]
                
                """Service time distribution are calculated as n times convolutions, where n is the number of clients scheduled."""
                self.system['v'][self.state] = convolve_n(self.parameters['s'].copy(), self.parameters['x'][self.state])[:(len(self.parameters['s']))]
                
                """The probablitity that the amount of work left in the system equals i just before state t ends equals the convolution of the probability that the amount of work left in the system equals i just before state t starts and the service time distribution."""
                self.system['p_plus'][self.state] = np.convolve(self.system['p_min'][self.state], self.system['v'][self.state])[:(len(self.parameters['s']))]


In [3]:
"""Simple test case"""

schedule = Schedule(x=np.array([1, 0, 2], dtype=np.int64), d=3, s=np.array(
    [0.1, 0.2, 0.3, 0.2, 0.15, 0.05, 0, 0, 0, 0], dtype=np.float64))
schedule.calculate_system_states(until=2)

print(schedule.system)


i = 0
{'p_min': array([[1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.8 , 0.15, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]]), 'v': array([[0.1   , 0.2   , 0.3   , 0.2   , 0.15  , 0.05  , 0.    , 0.    ,
        0.    , 0.    ],
       [1.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.01  , 0.04  , 0.1   , 0.16  , 0.2   , 0.19  , 0.15  , 0.09  ,
        0.0425, 0.015 ]]), 'p_plus': array([[0.1   , 0.2   , 0.3   , 0.2   , 0.15  , 0.05  , 0.    , 0.    ,
        0.    , 0.    ],
       [0.8   , 0.15  , 0.05  , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.01  , 0.04  , 0.1   , 0.16  , 0.2   , 0.19  , 0.15  , 0.09  ,
        0.0425, 0.015 ]])}


In [4]:
""""Test case with validation in spreadsheet
url: https://docs.google.com/spreadsheets/d/1_l9bMqEfLT2-TpZz3MrDFIid30ZsywTVH8Lzc5uHrGw/edit?usp=sharing"""
x = np.array([1, 0, 1, 0, 1, 0, 1], dtype=np.int64)
d = 5
service_times = range(0, 100)
l = 6
s = np.array([poisson.pmf(i, l) for i in service_times], dtype=np.float64)
schedule = Schedule(x=x, d=d, s=s)
schedule.calculate_system_states(until=2)
print(schedule.system)


i = 0
{'p_min': array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.000000

In [5]:
""""Test case with more than 1 clients in the system at the same time
url: https://docs.google.com/spreadsheets/d/1_l9bMqEfLT2-TpZz3MrDFIid30ZsywTVH8Lzc5uHrGw/edit?usp=sharing"""
x = np.array([1, 0, 2, 0, 1, 0, 1], dtype=np.int64)
d = 5
service_times = range(0, 200)
l = 6
s = np.array([poisson.pmf(i, l) for i in service_times], dtype=np.float64)
schedule = Schedule(x=x, d=d, s=s)
schedule.calculate_system_states(until=2)
print(schedule.system)


i = 0
{'p_min': array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.44567964, 0.16062314, 0.13767698, ..., 0.        , 0.        ,
        0.        ],
       [0.95737908, 0.02252896, 0.01126448, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]]), 'v': array([[2.47875218e-003, 1.48725131e-002, 4.46175392e-002, ...,
        4.89428276e-218, 1.48311599e-219, 4.47170650e-221],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [6.14421235e-006, 7.37305482e-005, 4.42383289e-004, ...,
        2.43686410e-161, 1.47688734e-162, 8.90585328e-164],
       ...,
       [0.0000000